In [ ]:
options(timeout = 600)
library(annotatepip)
setwd("/Users/hungm/Documents/development/annotatepip")

Loading required package: BiocManager

Bioconductor version '3.19' is out-of-date; the current release version '3.20'
  is available with R version '4.4'; see https://bioconductor.org/install

Loading required package: tidyverse

Warning message:
"package 'purrr' was built under R version 4.4.1"
Warning message:
"package 'lubridate' was built under R version 4.4.1"
-- Attaching core tidyverse packages ------------------------ tidyverse 2.0.0 --
v dplyr     1.1.4     v readr     2.1.5
v forcats   1.0.0     v stringr   1.5.1
v ggplot2   3.5.1     v tibble    3.2.1
v lubridate 1.9.4     v tidyr     1.3.1
v purrr     1.0.4     
-- Conflicts ------------------------------------------ tidyverse_conflicts() --
x dplyr::filter() masks stats::filter()
x dplyr::lag()    masks stats::lag()
i Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
Loading required package: biomaRt

Loading required package: OmnipathR

Warning message:
"package 'OmnipathR

## Retrieve biomRt database

In [33]:
import_biomart()
ls()

[1] "biomart_dict"      "human.omnipath.db" "human.regulon.db" 
[4] "human_biomart"     "mouse_biomart"     "omnipath.db"

In [3]:
biomart_dict %>% head(.)

,mouse_gene_symbol,mouse_chromosome,human_gene_symbol,human_chromosome
,<chr>,<chr>,<chr>,<chr>
1,mt-Nd4,MT,MT-ND4,MT
2,mt-Nd1,MT,MT-ND1,MT
3,mt-Nd5,MT,MT-ND5,MT
4,mt-Nd6,MT,MT-ND6,MT
5,mt-Nd2,MT,MT-ND2,MT
6,mt-Nd3,MT,MT-ND3,MT


In [33]:
setwd("/Users/hungm/Documents/development/annotatepip/")
write.table(biomart_dict, "inst/extdata/release-105.tsv", col.names = T, row.names = F, quote = F, sep = "\t")

## Build human omnipath database

In [81]:
human.omnipath.db <- import_omnipath_intercell()
##
human.omnipath.db <- human.omnipath.db %>%
    mutate(
        human_gene_symbol =  genesymbol,
        database_cs = ifelse(str_detect(database, "UniProt"), "UniProt", database)) %>%
    merge(biomart_dict, ., by = "human_gene_symbol", all = F) %>%
    dplyr::select(-c("genesymbol"))
human.omnipath.db %>% head(.)

,human_gene_symbol,mouse_gene_symbol,mouse_chromosome,human_chromosome,category,parent,database,scope,aspect,source,uniprot,entity_type,consensus_score,transmitter,receiver,secreted,plasma_membrane_transmembrane,plasma_membrane_peripheral,database_cs
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<chr>
1,A1BG,A1bg,15,19,intracellular,intracellular,ComPPI,generic,locational,resource_specific,P04217,protein,2,FALSE,FALSE,TRUE,FALSE,FALSE,ComPPI
2,A1BG,A1bg,15,19,intracellular,intracellular,OmniPath,generic,locational,composite,P04217,protein,2,FALSE,FALSE,TRUE,FALSE,FALSE,OmniPath
3,A1BG,A1bg,15,19,intracellular,intracellular,GO_Intercell,generic,locational,resource_specific,P04217,protein,2,FALSE,FALSE,TRUE,FALSE,FALSE,GO_Intercell
4,A1BG,A1bg,15,19,extracellular,extracellular,OmniPath,generic,locational,composite,P04217,protein,1,FALSE,FALSE,TRUE,FALSE,FALSE,OmniPath
5,A1BG,A1bg,15,19,secreted,secreted,HPA_secretome,generic,locational,resource_specific,P04217,protein,3,FALSE,FALSE,TRUE,FALSE,FALSE,HPA_secretome
6,A1BG,A1bg,15,19,secreted,secreted,OmniPath,generic,locational,composite,P04217,protein,3,FALSE,FALSE,TRUE,FALSE,FALSE,OmniPath


In [82]:
human.omnipath.db <- human.omnipath.db %>%
    dplyr::select(human_gene_symbol, human_chromosome, entity_type, category, parent, consensus_score, database_cs) %>%
    arrange(human_gene_symbol) %>%
    group_by(human_gene_symbol) %>%
    summarize(
        human_chromosome = unique(human_chromosome),
        entity_type = unique(entity_type),
        category = paste0(unique(category), collapse = ", "),
        parent = paste0(unique(parent), collapse = ", "),
        consensus_score = max(consensus_score, na.rm = T),
        is_cs = ifelse(str_detect(parent, "transmembrane|cell_surface|adhesion|junction"), T, F),
        database_cs = paste0(unique(database_cs), collapse = ", "),
        ) %>%
    ungroup()

human.omnipath.db %>% head(.)

human_gene_symbol,human_chromosome,entity_type,category,parent,consensus_score,is_cs,database_cs
<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<lgl>,<chr>
A1BG,19,protein,"intracellular, extracellular, secreted, receptor","intracellular, extracellular, secreted, receptor",3,FALSE,"ComPPI, OmniPath, GO_Intercell, HPA_secretome, LOCATE, UniProt, CellTalkDB"
A1CF,10,protein,intracellular,intracellular,3,FALSE,"LOCATE, OmniPath, GO_Intercell, ComPPI, UniProt"
A2M,12,protein,"intracellular, growth_factor_binder, extracellular, secreted, ecm_regulator, ligand_regulator","intracellular, ligand_regulator, extracellular, secreted, ecm_regulator",5,FALSE,"OmniPath, GO_Intercell, UniProt, HPA_secretome, Baccin2019, connectomeDB2020, Matrisome, LOCATE"
A3GALT2,1,protein,"intracellular, transmembrane","intracellular, transmembrane, transmembrane_predicted",3,TRUE,"GO_Intercell, UniProt, Phobius, OmniPath"
A4GALT,22,protein,"transmembrane_phobius, intracellular, transmembrane, transmembrane_sosui, transmembrane_predicted, transmembrane_tmhmm","transmembrane_predicted, intracellular, transmembrane",6,TRUE,"Almen2009, UniProt, ComPPI, OmniPath, Ramilowski_location, LOCATE, GO_Intercell, Phobius"
A4GNT,3,protein,"transmembrane, intracellular","transmembrane, intracellular",5,TRUE,"OmniPath, UniProt, Ramilowski_location, GO_Intercell, LOCATE"


In [83]:
human.regulon.db <- import_transcriptional_interactions(organism = 9606)
human.regulon.db %>% head(.)

source,target,source_genesymbol,target_genesymbol,is_directed,is_stimulation,is_inhibition,consensus_direction,consensus_stimulation,consensus_inhibition,sources,references,curation_effort,dorothea_level,n_references,n_resources
<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<dbl>,<chr>,<dbl>,<int>
P01106,O14746,MYC,TERT,1,1,0,1,1,0,CollecTRI;DoRothEA;ExTRI_CollecTRI;HTRI_CollecTRI;HTRIdb;HTRIdb_DoRothEA;KEGG-MEDICUS;NTNU.Curated_CollecTRI;Pavlidis2021_CollecTRI;RegNetwork_DoRothEA;TFactS_CollecTRI;TFactS_DoRothEA;TRED_DoRothEA;TRRUST_CollecTRI;TRRUST_DoRothEA,CollecTRI:10022128;CollecTRI:10491298;CollecTRI:10606235;CollecTRI:10637317;CollecTRI:10723141;CollecTRI:10786671;CollecTRI:10914736;CollecTRI:11274400;CollecTRI:11279234;CollecTRI:11287602;CollecTRI:11435602;CollecTRI:11606399;CollecTRI:11916966;CollecTRI:12044867;CollecTRI:12646176;CollecTRI:12695333;CollecTRI:12802280;CollecTRI:12807729;CollecTRI:12821782;CollecTRI:12941894;CollecTRI:12970752;CollecTRI:14519204;CollecTRI:14563552;CollecTRI:14611815;CollecTRI:14612409;CollecTRI:14744757;CollecTRI:15226182;CollecTRI:15371341;CollecTRI:15595642;CollecTRI:15655249;CollecTRI:15665468;CollecTRI:15958520;CollecTRI:16205635;CollecTRI:16287840;CollecTRI:16402390;CollecTRI:16489011;CollecTRI:16785237;CollecTRI:16880523;CollecTRI:17070091;CollecTRI:17141225;CollecTRI:17706770;CollecTRI:17934056;CollecTRI:17942561;CollecTRI:17967894;CollecTRI:18093727;CollecTRI:18158869;CollecTRI:18226852;CollecTRI:18365874;CollecTRI:18367227;CollecTRI:18586674;CollecTRI:18754863;CollecTRI:18818322;CollecTRI:19003324;CollecTRI:19047367;CollecTRI:19148473;CollecTRI:19360348;CollecTRI:19649288;CollecTRI:19762557;CollecTRI:19787269;CollecTRI:19806010;CollecTRI:19912441;CollecTRI:20454512;CollecTRI:20723213;CollecTRI:21132266;CollecTRI:21553143;CollecTRI:21627565;CollecTRI:21807113;CollecTRI:21855630;CollecTRI:21995449;CollecTRI:22207128;CollecTRI:23759068;CollecTRI:23860446;CollecTRI:24705139;CollecTRI:9988278;HTRIdb:11274400;HTRIdb:12695333,76,A,74,4
P17947,P02818,SPI1,BGLAP,1,1,0,1,1,0,CollecTRI;ExTRI_CollecTRI,CollecTRI:10022617,1,NA,1,1
COMPLEX:P15407_P17275,P05412,FOSL1_JUNB,JUN,1,1,0,1,1,0,CollecTRI;ExTRI_CollecTRI;NTNU.Curated_CollecTRI;TRRUST_CollecTRI,CollecTRI:10022869;CollecTRI:10037172;CollecTRI:10208431;CollecTRI:10366004;CollecTRI:11281649;CollecTRI:11736649;CollecTRI:12052888;CollecTRI:12209312;CollecTRI:12400010;CollecTRI:12903123;CollecTRI:12944913;CollecTRI:12972619;CollecTRI:1373300;CollecTRI:1419903;CollecTRI:14604893;CollecTRI:1594601;CollecTRI:16007074;CollecTRI:16679312;CollecTRI:16732317;CollecTRI:18094076;CollecTRI:1826351;CollecTRI:19437621;CollecTRI:19671687;CollecTRI:2169353;CollecTRI:22180088;CollecTRI:22658828;CollecTRI:23125217;CollecTRI:2548844;CollecTRI:3142689;CollecTRI:7631795;CollecTRI:7775415;CollecTRI:7894465;CollecTRI:7988742;CollecTRI:8065332;CollecTRI:8092982;CollecTRI:8114693;CollecTRI:8143767;CollecTRI:8453101;CollecTRI:8530413;CollecTRI:8597534;CollecTRI:8617996;CollecTRI:8797597;CollecTRI:8892460;CollecTRI:8947050;CollecTRI:9502786;CollecTRI:9687385;CollecTRI:9731701;CollecTRI:9743233;CollecTRI:9834073,49,NA,49,1
COMPLEX:P01100_P05412,P05412,FOS_JUN,JUN,1,1,0,1,1,0,CollecTRI;ExTRI_CollecTRI;NTNU.Curated_CollecTRI;TRRUST_CollecTRI,CollecTRI:10022869;CollecTRI:10037172;CollecTRI:10208431;CollecTRI:10366004;CollecTRI:11281649;CollecTRI:11736649;CollecTRI:12052888;CollecTRI:12209312;CollecTRI:12400010;CollecTRI:12903123;CollecTRI:12944913;CollecTRI:12972619;CollecTRI:1373300;CollecTRI:1419903;CollecTRI:14604893;CollecTRI:1594601;CollecTRI:16007074;CollecTRI:16679312;CollecTRI:16732317;CollecTRI:18094076;CollecTRI:1826351;CollecTRI:19437621;CollecTRI:19671687;CollecTRI:2169353;CollecTRI:22180088;CollecTRI:22658828;CollecTRI:23125217;CollecTRI:2548844;CollecTRI:3142689;CollecTRI:7631795;CollecTRI:7775415;CollecTRI:7894465;CollecTRI:7988742;CollecTRI:8065332;CollecTRI:8092982;CollecTRI:8114693;CollecTRI:8143767;CollecTRI:8453101;CollecTR

In [84]:
human.regulon.db <- human.regulon.db %>%
    group_by(source_genesymbol) %>%
    separate_rows(source_genesymbol, sep = "_") %>%
    separate_rows(sources, sep = ";") %>%
    mutate(human_gene_symbol = source_genesymbol) %>%
    group_by(human_gene_symbol) %>%
    summarize(database_tf = paste0(unique(sources), collapse = ";"))

human.regulon.db %>% head(.)

human_gene_symbol,database_tf
<chr>,<chr>
AATF,SIGNOR
ABL1,CollecTRI;TRRUST_CollecTRI;GOA_CollecTRI
ACTL6A,SIGNOR
ADNP,CollecTRI;SIGNOR_CollecTRI
ADNP2,CollecTRI;ExTRI_CollecTRI
AEBP1,CollecTRI;ExTRI_CollecTRI;GOA_CollecTRI


In [85]:
human.omnipath.db <- human.omnipath.db %>%
    mutate(is_tf = ifelse(human_gene_symbol %in% human.regulon.db$human_gene_symbol, T, F)) %>%
    merge(., human.regulon.db, by = "human_gene_symbol", all.x = T)

human.omnipath.db %>% head(.)

,human_gene_symbol,human_chromosome,entity_type,category,parent,consensus_score,is_cs,database_cs,is_tf,database_tf
,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<lgl>,<chr>,<lgl>,<chr>
1,A1BG,19,protein,"intracellular, extracellular, secreted, receptor","intracellular, extracellular, secreted, receptor",3,FALSE,"ComPPI, OmniPath, GO_Intercell, HPA_secretome, LOCATE, UniProt, CellTalkDB",FALSE,NA
2,A1CF,10,protein,intracellular,intracellular,3,FALSE,"LOCATE, OmniPath, GO_Intercell, ComPPI, UniProt",FALSE,NA
3,A2M,12,protein,"intracellular, growth_factor_binder, extracellular, secreted, ecm_regulator, ligand_regulator","intracellular, ligand_regulator, extracellular, secreted, ecm_regulator",5,FALSE,"OmniPath, GO_Intercell, UniProt, HPA_secretome, Baccin2019, connectomeDB2020, Matrisome, LOCATE",FALSE,NA
4,A3GALT2,1,protein,"intracellular, transmembrane","intracellular, transmembrane, transmembrane_predicted",3,TRUE,"GO_Intercell, UniProt, Phobius, OmniPath",FALSE,NA
5,A4GALT,22,protein,"transmembrane_phobius, intracellular, transmembrane, transmembrane_sosui, transmembrane_predicted, transmembrane_tmhmm","transmembrane_predicted, intracellular, transmembrane",6,TRUE,"Almen2009, UniProt, ComPPI, OmniPath, Ramilowski_location, LOCATE, GO_Intercell, Phobius",FALSE,NA
6,A4GNT,3,protein,"transmembrane, intracellular","transmembrane, intracellular",5,TRUE,"OmniPath, UniProt, Ramilowski_location, GO_Intercell, LOCATE",FALSE,NA


In [86]:
write.table(human.omnipath.db, "inst/extdata/release-105_human_omnipath_db.tsv", col.names = T, row.names = F, quote = F, sep = "\t")

## Build mouse omnipath database

In [90]:
mouse.omnipath.db <- import_omnipath_intercell(organism = 10090)
##
mouse.omnipath.db <- mouse.omnipath.db %>%
    mutate(
        human_gene_symbol =  genesymbol,
        database_cs = ifelse(str_detect(database, "UniProt"), "UniProt", database)) %>%
    merge(biomart_dict, ., by = "human_gene_symbol", all = F) %>%
    dplyr::select(-c("genesymbol"))
mouse.omnipath.db %>% head(.)

,human_gene_symbol,mouse_gene_symbol,mouse_chromosome,human_chromosome,category,parent,database,scope,aspect,source,uniprot,entity_type,consensus_score,transmitter,receiver,secreted,plasma_membrane_transmembrane,plasma_membrane_peripheral,database_cs
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<chr>
1,A1BG,A1bg,15,19,intracellular,intracellular,ComPPI,generic,locational,resource_specific,P04217,protein,2,FALSE,FALSE,TRUE,FALSE,FALSE,ComPPI
2,A1BG,A1bg,15,19,intracellular,intracellular,OmniPath,generic,locational,composite,P04217,protein,2,FALSE,FALSE,TRUE,FALSE,FALSE,OmniPath
3,A1BG,A1bg,15,19,intracellular,intracellular,GO_Intercell,generic,locational,resource_specific,P04217,protein,2,FALSE,FALSE,TRUE,FALSE,FALSE,GO_Intercell
4,A1BG,A1bg,15,19,extracellular,extracellular,OmniPath,generic,locational,composite,P04217,protein,1,FALSE,FALSE,TRUE,FALSE,FALSE,OmniPath
5,A1BG,A1bg,15,19,secreted,secreted,HPA_secretome,generic,locational,resource_specific,P04217,protein,3,FALSE,FALSE,TRUE,FALSE,FALSE,HPA_secretome
6,A1BG,A1bg,15,19,secreted,secreted,OmniPath,generic,locational,composite,P04217,protein,3,FALSE,FALSE,TRUE,FALSE,FALSE,OmniPath


In [93]:
mouse.omnipath.db <- mouse.omnipath.db %>%
    dplyr::select(mouse_gene_symbol, mouse_chromosome, entity_type, category, parent, consensus_score, database_cs) %>%
    arrange(mouse_gene_symbol) %>%
    group_by(mouse_gene_symbol) %>%
    summarize(
        mouse_gene_symbol = unique(mouse_gene_symbol),
        mouse_chromosome = unique(mouse_chromosome),
        entity_type = unique(entity_type),
        category = paste0(unique(category), collapse = ", "),
        parent = paste0(unique(parent), collapse = ", "),
        consensus_score = max(consensus_score, na.rm = T),
        is_cs = ifelse(str_detect(parent, "transmembrane|cell_surface|adhesion|junction"), T, F),
        database_cs = paste0(unique(database_cs), collapse = ", "),
        ) %>%
    ungroup() %>%
    filter(mouse_gene_symbol != "")

mouse.omnipath.db %>% head(.)
mouse.omnipath.db %>% dim(.)

mouse_gene_symbol,mouse_chromosome,entity_type,category,parent,consensus_score,is_cs,database_cs
<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<lgl>,<chr>
0610010K14Rik,11,protein,intracellular,intracellular,1,FALSE,"ComPPI, OmniPath"
0610030E20Rik,6,protein,intracellular,intracellular,1,FALSE,"ComPPI, OmniPath"
0610040J01Rik,5,protein,intracellular,intracellular,1,FALSE,"OmniPath, ComPPI"
1110032A03Rik,9,protein,intracellular,intracellular,3,FALSE,"OmniPath, GO_Intercell, ComPPI, UniProt"
1110032F04Rik,3,protein,"cell_surface, transmembrane","cell_surface, transmembrane, transmembrane_predicted",4,TRUE,"Surfaceome, OmniPath, Ramilowski_location, UniProt, Phobius"
1110038F14Rik,15,protein,intracellular,intracellular,1,FALSE,"ComPPI, OmniPath"


[1] 18579     8

In [94]:
mouse.regulon.db <- import_transcriptional_interactions(organism = 10090)
mouse.regulon.db %>% head(.)

source,target,source_genesymbol,target_genesymbol,is_directed,is_stimulation,is_inhibition,consensus_direction,consensus_stimulation,consensus_inhibition,sources,references,curation_effort,dorothea_level,n_references,n_resources
<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<dbl>,<chr>,<dbl>,<int>
P01108,O70372,Myc,Tert,1,1,0,1,1,0,CollecTRI;DoRothEA;ExTRI_CollecTRI;HTRI_CollecTRI;HTRIdb;HTRIdb_DoRothEA;KEGG-MEDICUS;NTNU.Curated_CollecTRI;Pavlidis2021_CollecTRI;RegNetwork_DoRothEA;TFactS_CollecTRI;TFactS_DoRothEA;TRED_DoRothEA;TRRUST_CollecTRI;TRRUST_DoRothEA,CollecTRI:10022128;CollecTRI:10491298;CollecTRI:10606235;CollecTRI:10637317;CollecTRI:10723141;CollecTRI:10786671;CollecTRI:10914736;CollecTRI:11274400;CollecTRI:11279234;CollecTRI:11287602;CollecTRI:11435602;CollecTRI:11606399;CollecTRI:11916966;CollecTRI:12044867;CollecTRI:12646176;CollecTRI:12695333;CollecTRI:12802280;CollecTRI:12807729;CollecTRI:12821782;CollecTRI:12941894;CollecTRI:12970752;CollecTRI:14519204;CollecTRI:14563552;CollecTRI:14611815;CollecTRI:14612409;CollecTRI:14744757;CollecTRI:15226182;CollecTRI:15371341;CollecTRI:15595642;CollecTRI:15655249;CollecTRI:15665468;CollecTRI:15958520;CollecTRI:16205635;CollecTRI:16287840;CollecTRI:16402390;CollecTRI:16489011;CollecTRI:16785237;CollecTRI:16880523;CollecTRI:17070091;CollecTRI:17141225;CollecTRI:17706770;CollecTRI:17934056;CollecTRI:17942561;CollecTRI:17967894;CollecTRI:18093727;CollecTRI:18158869;CollecTRI:18226852;CollecTRI:18365874;CollecTRI:18367227;CollecTRI:18586674;CollecTRI:18754863;CollecTRI:18818322;CollecTRI:19003324;CollecTRI:19047367;CollecTRI:19148473;CollecTRI:19360348;CollecTRI:19649288;CollecTRI:19762557;CollecTRI:19787269;CollecTRI:19806010;CollecTRI:19912441;CollecTRI:20454512;CollecTRI:20723213;CollecTRI:21132266;CollecTRI:21553143;CollecTRI:21627565;CollecTRI:21807113;CollecTRI:21855630;CollecTRI:21995449;CollecTRI:22207128;CollecTRI:23759068;CollecTRI:23860446;CollecTRI:24705139;CollecTRI:9988278;HTRIdb:11274400;HTRIdb:12695333,76,A,74,4
P17433,P86546,Spi1,Bglap,1,1,0,1,1,0,CollecTRI;ExTRI_CollecTRI,CollecTRI:10022617,1,NA,1,1
P17433,P54615,Spi1,Bglap3,1,1,0,1,1,0,CollecTRI;ExTRI_CollecTRI,CollecTRI:10022617,1,NA,1,1
P17433,P86547,Spi1,Bglap2,1,1,0,1,1,0,CollecTRI;ExTRI_CollecTRI,CollecTRI:10022617,1,NA,1,1
Q8BUN5,P05627,Smad3,Jun,1,1,0,1,1,0,CollecTRI;DoRothEA;ExTRI_CollecTRI;NTNU.Curated_CollecTRI;RegNetwork_DoRothEA;TFactS_CollecTRI;TFactS_DoRothEA;TRED_DoRothEA;TRRUST_CollecTRI;TRRUST_DoRothEA,CollecTRI:10022869;CollecTRI:12374795;DoRothEA:10022869,3,A,2,2
P97471,P05627,Smad4,Jun,1,1,0,1,1,0,CollecTRI;DoRothEA;ExTRI_CollecTRI;NTNU.Curated_CollecTRI;RegNetwork_DoRothEA;TFactS_CollecTRI;TFactS_DoRothEA;TRED_DoRothEA;TRRUST_CollecTRI;TRRUST_DoRothEA,CollecTRI:10022869;CollecTRI:12374795;DoRothEA:10022869,3,A,2,2


In [95]:
mouse.regulon.db <- mouse.regulon.db %>%
    group_by(source_genesymbol) %>%
    separate_rows(source_genesymbol, sep = "_") %>%
    separate_rows(sources, sep = ";") %>%
    mutate(mouse_gene_symbol = source_genesymbol) %>%
    group_by(mouse_gene_symbol) %>%
    summarize(database_tf = paste0(unique(sources), collapse = ";"))

mouse.regulon.db %>% head(.)

mouse_gene_symbol,database_tf
<chr>,<chr>
Aatf,SIGNOR
Abl1,CollecTRI;TRRUST_CollecTRI;GOA_CollecTRI
Adnp,CollecTRI;SIGNOR_CollecTRI
Adnp2,CollecTRI;ExTRI_CollecTRI
Aebp1,CollecTRI;ExTRI_CollecTRI;GOA_CollecTRI
Aebp2,CollecTRI;GOA_CollecTRI


In [96]:
mouse.omnipath.db <- mouse.omnipath.db %>%
    merge(., mouse.regulon.db, by = "mouse_gene_symbol",all.x = T)

mouse.omnipath.db %>% head(.)

,mouse_gene_symbol,mouse_chromosome,entity_type,category,parent,consensus_score,is_cs,database_cs,database_tf
,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<lgl>,<chr>,<chr>
1,0610010K14Rik,11,protein,intracellular,intracellular,1,FALSE,"ComPPI, OmniPath",NA
2,0610030E20Rik,6,protein,intracellular,intracellular,1,FALSE,"ComPPI, OmniPath",NA
3,0610040J01Rik,5,protein,intracellular,intracellular,1,FALSE,"OmniPath, ComPPI",NA
4,1110032A03Rik,9,protein,intracellular,intracellular,3,FALSE,"OmniPath, GO_Intercell, ComPPI, UniProt",NA
5,1110032F04Rik,3,protein,"cell_surface, transmembrane","cell_surface, transmembrane, transmembrane_predicted",4,TRUE,"Surfaceome, OmniPath, Ramilowski_location, UniProt, Phobius",NA
6,1110038F14Rik,15,protein,intracellular,intracellular,1,FALSE,"ComPPI, OmniPath",NA


In [97]:
write.table(mouse.omnipath.db, "inst/extdata/release-105_mouse_omnipath_db.tsv", col.names = T, row.names = F, quote = F, sep = "\t")

In [98]:
sessionInfo()

R version 4.4.0 (2024-04-24)
Platform: aarch64-apple-darwin20
Running under: macOS Sonoma 14.6.1

Matrix products: default
BLAS:   /Library/Frameworks/R.framework/Versions/4.4-arm64/Resources/lib/libRblas.0.dylib 
LAPACK: /Library/Frameworks/R.framework/Versions/4.4-arm64/Resources/lib/libRlapack.dylib;  LAPACK version 3.12.0

locale:
[1] C

time zone: Europe/London
tzcode source: internal

attached base packages:
[1] stats     graphics  grDevices utils     datasets  methods   base     

other attached packages:
 [1] annotatepip_0.0.0   OmnipathR_3.12.4    biomaRt_2.60.1     
 [4] lubridate_1.9.4     forcats_1.0.0       stringr_1.5.1      
 [7] dplyr_1.1.4         purrr_1.0.4         readr_2.1.5        
[10] tidyr_1.3.1         tibble_3.2.1        ggplot2_3.5.1      
[13] tidyverse_2.0.0     BiocManager_1.30.22

loaded via a namespace (and not attached):
 [1] tidyselect_1.2.1        IRdisplay_1.1           blob_1.2.4             
 [4] filelock_1.0.3          Biostrings_2.72.1       fas